#### First attempt to convert ds004158 as BIDS valid eyetracking dataset

#### To do
- [ ] get right file list in sourcedata
- [ ] execute edf2asc command
- [ ] convert to TSV.GZ the asc file

In [1]:
import glob
from bids import BIDSLayout
import numpy as np
import pandas as pd
import subprocess
from subprocess import DEVNULL, STDOUT

# define data columns
eye_column_names = ['eye_timestamp', 'eye1_x_coordinate', 'eye1_y_coordinate']
usecols = (0, 1, 2)

# define dataset folder
ds_folder = '../ds004158'

In [2]:
# get bids layout
layout = BIDSLayout(ds_folder)

In [3]:
# get subjects id
subject_ids = layout.get(return_type='id', target='subject')

In [ ]:
for subject_id in subject_ids:

    # convert to asc
    source_edf_fns = glob.glob("{}/sourcedata/sub-{}/ses-01/func/*.edf".format(ds_folder,subject_id))
    for source_edf_fn in source_edf_fns:
        cmd='edf2asc {} -s -miss -1.0 -y'.format(source_edf_fn)
        subprocess.run(cmd, shell=True, stdout=DEVNULL, stderr=STDOUT)

    # load asc
    source_asc_fns = glob.glob("{}/sourcedata/sub-{}/ses-01/func/*.asc".format(ds_folder,subject_id))

    # get nifti
    nifti_fns = layout.get(subject=subject_id, extension='nii.gz', suffix='bold', return_type='filename')
    
    for source_asc_fn, nifti_fn in zip(source_asc_fns,nifti_fns):
        
        bids_tsvgz_fn = "{}_eyetrack.tsv.gz".format(nifti_fn[:-12])
        print("saving {}".format(bids_tsvgz_fn))
        
        eye_data_run = np.genfromtxt(source_asc_fn, usecols=usecols)
        df = pd.DataFrame(eye_data_run, columns=eye_column_names)
        df.to_csv(bids_tsvgz_fn, sep='\t', index=False, compression='gzip')
        

saving /Users/martinszinte/Desktop/eye2bids/../ds004158/sub-01/ses-01/func/sub-01_ses-01_task-rest_run-01_eyetrack.tsv.gz
saving /Users/martinszinte/Desktop/eye2bids/../ds004158/sub-01/ses-01/func/sub-01_ses-01_task-rest_run-02_eyetrack.tsv.gz
saving /Users/martinszinte/Desktop/eye2bids/../ds004158/sub-02/ses-01/func/sub-02_ses-01_task-rest_run-01_eyetrack.tsv.gz
saving /Users/martinszinte/Desktop/eye2bids/../ds004158/sub-02/ses-01/func/sub-02_ses-01_task-rest_run-02_eyetrack.tsv.gz
saving /Users/martinszinte/Desktop/eye2bids/../ds004158/sub-03/ses-01/func/sub-03_ses-01_task-rest_run-01_eyetrack.tsv.gz
saving /Users/martinszinte/Desktop/eye2bids/../ds004158/sub-03/ses-01/func/sub-03_ses-01_task-rest_run-02_eyetrack.tsv.gz
